**Текущие задачи**:
- Добавить обработку k для числа просмотров у хабра
- Реализация опроса новых данных о свежих статьях
- Описать `requirements` или сделать Docker-образ
- Выгружать датасет из памяти после того, как он был считан. Составлять новый датасет, выводить в csv и конкатенировать сами csv.

# Парсер

Реализованы парсеры данных, собраны тексты и метаинформация статей следующих ресурсов:
- Хабрахабр https://habr.com/
- Типичный программист https://tproger.ru/

Также реализован механизм докачивания новых статей, чтобы учесть появление новых технологий. Чтобы не создавать повышенную нагрузку на серверы ресурсов и не вызывать их подозрений, опрашивание ресурсов происходит поочередно. Методы парсеров содержатся внутри класса `Parser`, методы оркестровки (планирование и общий пайплайн вызова парсеров) – в классе `ParserComposer`.

Системные требования:
- Каталог с датасетами
- Дравйвер `/usr/bin/chromedriver`

In [1]:
# standard library
import random
import datetime
import pickle
import os
import re
import uuid

# conections
import requests
from requests.exceptions import ConnectionError

# tor network for private parsing with privoxy and stem
# https://gist.github.com/KhepryQuixote/46cf4f3b999d7f658853
from stem import Signal
from stem.control import Controller

# for tor network IP changing
with Controller.from_port(port = 9051) as controller:
    controller.authenticate('07011951')
    controller.signal(Signal.NEWNYM)

headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.73.11 (KHTML, like Gecko) Version/7.0.1 Safari/537.73.11'
}

# parsers libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# data science 
import pandas as pd
from tqdm.notebook import tqdm, trange

sources = {
    'tproger':
        {
            'base_url': 'https://tproger.ru/',
            'first_page_url': 'https://tproger.ru/page/1/',
            'find_all_args': ('a', {'class':'article-link'})
        },
    'habr':
        {
            'base_url': 'https://habr.com/',
            'first_page_url': 'https://habr.com/ru/page1/',
            'find_all_args': ('a', {'class':'post__title_link'})
        }
}

proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}

# ссылки на новые статьи, которые не были обработаны ранее
with open('not_processed_urls.pickle', 'rb') as f:
    nurls = pickle.load(f)

DATASETS_PATH = '/home/leo/DATASETS'

# for source_name in sources:
#     sources[source_name]['df'] = pd.read_csv(f'{DATASETS_PATH}/{source_name}.csv',
#                                              index_col=0,
#                                              parse_dates=['post_time', 'parse_time'])

In [14]:
def k_to_num(s):
    if type(s) is str:
        s = s.replace(',', '.')
        if 'k' in s:
            s = s.replace('k', 'e3')
    return round(float(s))

class Parser:
    def __init__(self, source_name):
        self.source_name = source_name
        self.source = sources[source_name]
        self.df = pd.read_csv(f"{DATASETS_PATH}/{source_name}.csv",
                         index_col=0,
                         parse_dates=['post_time', 'parse_time'])
    
    def delta_dates_days(self):
        today_date = datetime.datetime.now()
        last_parse_date = self.df.post_time.max()
        today_date = today_date.replace(tzinfo=last_parse_date.tz)
        delta_dates = today_date - last_parse_date
        return delta_dates.days
    
    def check_new_articles(self):
        '''Проверяет, появились ли на ресурсе новые публикации,
        и если появились, заносит их в план на скачивание.'''
        all_new_urls = nurls[self.source_name]
        page_count = 1
        source = self.source

        attempts_counts = 0
        while True:
            try:
                proxies = {
                    'http': 'socks5h://127.0.0.1:9050',
                    'https': 'socks5h://127.0.0.1:9050'
                }
                page_url = source['first_page_url'].replace('1', str(page_count))
                page = requests.get(page_url,
                                    headers=headers,
                                    proxies=proxies,
                                    stream=False)
                soup = BeautifulSoup(page.text, 'html.parser')
                urls = {url['href'] for url in soup.find_all(*source['find_all_args'])}
                if not urls:
                    # the thread processes the case when captcha and no urls
                    print("Changing proxy because of no urls or CAPTCHA...")
                    attempts_counts += 1
                    if attempts_counts >= 2:
                        print(f"Data parsing of {self.source_name} is stopped at {page_url}.")
                        break
                    proxies = None
                    continue
                old_urls = set(self.df.index.to_list())
                new_urls = urls - old_urls
                all_new_urls = all_new_urls | new_urls
                nurls[self.source_name] = all_new_urls
                if len(new_urls) > 0:
                    print(f"{len(new_urls)} urls are collected from {page_url}")
                if (len(new_urls) == 0) or (len(urls) == 0):    
                    print(f"{len(all_new_urls)} new urls are saved for {source['base_url']}")
                    break
                else:
                    page_count += 1
            except ConnectionError:
                print("Problems with connection...")
                continue
        with open('not_processed_urls.pickle', 'wb') as f:
            pickle.dump(nurls, f)


    def save_new_data(self, url, data, full_text):
        '''Сохраняет новые данные в датафрейм, а текст -- в отдельный файл'''
        data['filename'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, url))
        new_row = pd.DataFrame(data, index=[url])
        if url not in self.df:
            self.df = self.df.append(new_row)
        else:
            self.df.iloc[url] = new_row
        
        filepath = f"{DATASETS_PATH}/{self.source_name}/{data['filename']}"
        if not os.path.exists(filepath):
            with open(filepath, 'w') as f:
                full_text = re.sub('\n+', '\n\n', full_text).strip()
                f.write(full_text)
                #print(f"File {self.source_name}/{data['filename']} is saved.")
        else:
            pass
            #print(f"File {self.source_name}/{data['filename']} is already exist.")
            
    def save_df(self):
        self.df.to_csv(f"{DATASETS_PATH}/{self.source_name}.csv")
            
    def concat_csv(self):
        '''
        Соединяет старый csv и тот, что получен из новых данных.
        '''
        pass
    

class HabrParser(Parser):
    '''Парсер для сайта habr.com'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        proxies = {
            'http': 'socks5h://127.0.0.1:9050',
            'https': 'socks5h://127.0.0.1:9050'
        }
        page = requests.get(url, headers=headers, proxies=proxies, stream=False)
        soup = BeautifulSoup(page.text, 'html.parser')
        data['title'] = soup.select_one('h1').text.strip()
        try:
            date_time = soup.select_one('.post__time').get('data-time_published')
            data['post_time'] = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%MZ')
        except AttributeError:
            date_time = None
        try:
            data['views_num'] = k_to_num(soup.select_one('.post-stats__views-count').text.strip())
        except AttributeError:
            data['views_num'] = 0
        try:
            data['likes_num'] = k_to_num(soup.select_one('.js-post-vote').text.strip().replace('–', '-'))
        except AttributeError:
            data['likes_num'] = 0
        try:
            data['favs_num'] = k_to_num(soup.select_one('.bookmark__counter').text)
        except AttributeError:
            data['favs_num'] = 0
        try:
            data['comments_num'] = k_to_num(soup.select_one('.post-stats__comments-count').text)
        except AttributeError:
            data['comments_num'] = 0
        data['parse_time'] = datetime.datetime.now()
        try:
            full_text = soup.select_one('.post__body_full').text
        except AttributeError:
            full_text = ""
        self.save_new_data(url, data, full_text)


class TprogerParser(Parser):
    '''Парсер для сайта tproger.ru'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        try:
            driver = webdriver.Chrome('/usr/bin/chromedriver')
            driver.get(url)
            page = driver.page_source
            element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.post-views-count'))
            WebDriverWait(driver, timeout=10).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
        finally:
            driver.quit()

        soup = BeautifulSoup(page, 'html.parser')
        data['title'] = soup.h1.text.strip()
        data['post_time'] = datetime.datetime.fromisoformat(soup.select_one('time')['content'])
        paragraphs = soup.find_all('div', {'class':'entry-content'})[0].find_all('p')
        try:
            data['short_text'] = paragraphs[0].text
        except IndexError:
            data['short_text'] = ''
        try:
            data['views_num'] = soup.select_one('.post-views-count').text
        except AttributeError:
            data['views_num'] = 0
        full_text = "\n\n".join(p.text for p in paragraphs[:-1])
        self.save_new_data(url, data, full_text)    
        
        
class ParserComposer:
    def __init__(self):
        pass
    
    def scheduler(self):
        '''
        Сортирует задачи таким образом, чтобы
        равномерно распределить запросы к внешним ресурсам.
        Порядок проверки:
        1) загружаем те статьи, что не были обработаны по каким-то причинам ранее,
        2) проверяем, не вышли ли новые статьи
        3) обновляем число просмотров статей, вышедших в последний месяц (post_time)
        4) обновляем число просмотров статей, проверявшихся > месяца назад (parse_time), не относящихся к п. 2
        '''
        pass
    
    def pipeline(self):
        '''Проверяем, вышли ли новые статьи. Планируем загрузки. Обновляем датасеты.'''
        pass

In [15]:
habr_parser = HabrParser('habr')
#habr_parser.check_new_articles()

In [16]:
tproger_parser = TprogerParser('tproger')
#tproger_parser.check_new_articles()

In [5]:
# for source_name in sources:
#     eval(f"{source_name}_parser.save_df()")
#     nurls[source_name] -= set(eval(f"{source_name}_parser.df.index"))

In [ ]:
def load_new_articles(nurls):
    pairs = [(key, item) for key in nurls for item in nurls[key]]
    random.shuffle(pairs)
    try:
        for pair in tqdm(pairs):
            try:
                eval(f"{pair[0]}_parser.update('{pair[1]}')")
            except ConnectionError:
                continue
    finally:
        for source_name in sources:
            eval(f"{source_name}_parser.save_df()")
            nurls[source_name] -= set(eval(f"{source_name}_parser.df.index"))
        with open('not_processed_urls.pickle', 'wb') as f:
            pickle.dump(nurls, f)

load_new_articles(nurls)

  0%|          | 0/55 [00:00<?, ?it/s]

File habr/5809bbb4-776e-5ef3-8640-27d1dabfb15e is saved.
File habr/b22a662d-7e2f-5dc2-9f29-7eab040e0652 is already exist.
File habr/70f71d1a-13be-5da4-8296-9668d07b07b0 is saved.
File habr/77685b09-6cf8-5feb-be27-cb73954dcad4 is saved.
File habr/00602bfa-6607-59a6-a4a8-ac9e6e73f2c9 is already exist.
File habr/ef781324-c1b7-501f-98e1-4153a4305bcf is already exist.
File habr/84cd27f8-7857-5f4a-80d2-97feecb87e08 is already exist.
File habr/555042f9-4f71-548f-8981-e4a293cd3767 is already exist.
File habr/420897a9-f116-5bc2-99ad-b8c164c780b3 is saved.
File habr/2dae4488-7d4d-5488-92e0-29c6a3e83d6e is saved.
File habr/cfa77db7-704a-507e-a829-22c1533be441 is saved.
File habr/19d56970-d1a7-5afc-b4f3-495166e865da is saved.
File habr/82bcc965-50b8-5d37-a209-0719ee6d86c2 is saved.
File habr/af55bf34-5758-568e-8def-50e3950a7049 is saved.
File habr/b7c2f44f-b92b-504b-b851-a03a8561847d is saved.
File habr/fc2dcbcc-af42-5ded-921d-962cef5ae74a is saved.
File habr/d29b10b0-46b8-5b5f-ba47-d0206fe373a4 i

In [17]:
with open('not_processed_urls.pickle', 'wb') as f:
    pickle.dump(nurls, f)

In [18]:
for source_name in sources:
    eval(f"{source_name}_parser.save_df()")
    nurls[source_name] -= set(eval(f"{source_name}_parser.df.index"))

In [19]:
habr_parser.check_new_articles()

20 urls are collected from https://habr.com/ru/page1/
5 urls are collected from https://habr.com/ru/page2/
4 urls are collected from https://habr.com/ru/page3/
6 urls are collected from https://habr.com/ru/page4/
3 urls are collected from https://habr.com/ru/page5/
6 urls are collected from https://habr.com/ru/page6/
2 urls are collected from https://habr.com/ru/page7/
55 new urls are saved for https://habr.com/
